# Daniels parts analysis

In [ ]:
import  pandas
import re
import pygal
from IPython.display import SVG, display

In [ ]:
# Note: data downloaded 17/March/2024
data = pandas.read_excel("./source/Amplifier SINAD list ASR + 3rd parties.xlsx", 
                  engine="openpyxl",
                  sheet_name="data")

In [ ]:
data.columns

In [ ]:
#  rename columns

data = data[
    [
        "Name", "SINAD (dB)\n(5W, 4 ohms)", "Price (USD)", "Power (W)\n(1% THD+N, 4 ohms)"
    ]
]
data.columns = [
    "name",  "sinad_5w_4ohm", "price_usd", "thd_1pc_4ohm"
]

In [ ]:
data.info()

In [ ]:
describe = data.describe()
describe

In [ ]:
###  can use these quartiles to divide thd column  into 4 categories

first_quartile_thd_bound = describe.loc["25%", "thd_1pc_4ohm"]
second_quartile_thd_bound = describe.loc["50%", "thd_1pc_4ohm"]
third_quartile_thd_bound = describe.loc["75%", "thd_1pc_4ohm"]

In [ ]:
def parse_thd_category(thd):
    if (thd < first_quartile_thd_bound):
        return f"0 - {first_quartile_thd_bound}"
    elif (thd < second_quartile_thd_bound):
        return f"{first_quartile_thd_bound} - {second_quartile_thd_bound}"
    elif (thd < third_quartile_thd_bound):
        return f"{second_quartile_thd_bound} - {third_quartile_thd_bound}"
    return f"{third_quartile_thd_bound} and above"

In [ ]:
def parse_usd(price):
    parsed = re.match("^\d+",  str(price))
    if (parsed is None):
        return -1
    return float(price)

In [ ]:
def display_and_export_ranked_horizontal_bar_chart(chart_data, chart_title, x_axis, y_axis, export_location):
    values = list(zip(chart_data[y_axis], chart_data[x_axis]))
    chart = pygal.HorizontalBar(legend_at_bottom=True,  
                                human_readable=True, 
                                print_values=True,
                               print_labels=True)
    chart.title = chart_title

    for c in values:
        chart.add(c[0], round(c[1], 2))

    chart.render_to_file(export_location)
    display(SVG(chart.render(disable_xml_declaration=True)))

In [ ]:
# convert  the price column to float values and get rid of entries without a price
data["price_usd"] = data["price_usd"].apply(parse_usd)
data = data[data["price_usd"] > 0]

In [ ]:
data["thd_category"] = data["thd_1pc_4ohm"].apply(parse_thd_category)

In [ ]:
data["price_to_sinad"] = data.apply(lambda x: x.sinad_5w_4ohm / x.price_usd, axis=1)
data["price_to_thd_1pc"] = data.apply(lambda x: x.thd_1pc_4ohm / x.price_usd, axis=1)

In [ ]:
data.shape

## THD @ 1% 4ohm analysis

### Top 10 price to thd 1% regardless of price

In [ ]:
subset = data.sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

In [ ]:
display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% regardless of price',
    export_location='./chart-exports/top-10-price_to_thd_1pc-unfiltered.svg'
)

### Top 10 price to thd 1%,  under 125USD

In [ ]:
subset = data[data["price_usd"] < 125].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

In [ ]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $125 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-125-usd.svg'
)

### Top 10 price to thd 1%,  under 250USD

In [ ]:
subset = data[data["price_usd"] < 250].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

In [ ]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $250 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-250-usd.svg'
)

### Top 10 price to thd 1%,  under 500USD

In [ ]:
subset = data[data["price_usd"] < 500].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

In [ ]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $500 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-500-usd.svg'
)

### Top 10 price to thd 1%,  under 1000USD

In [ ]:
subset = data[data["price_usd"] < 1000].sort_values("price_to_thd_1pc", ascending=False).head(10)
subset

In [ ]:

display_and_export_ranked_horizontal_bar_chart(
    chart_data=subset,
    x_axis='price_to_thd_1pc',
    y_axis='name',
    chart_title='Top 10 price to thd 1% under $1000 USD',
    export_location='./chart-exports/top-10-price_to_thd_1pc-under-1000-usd.svg'
)

### Top 10 price to thd 1%, in 0-115.5 (thd_1pc_4ohm) category

In [ ]:
data[
    data["thd_category"] == "0 - 115.5"
].sort_values("price_to_thd_1pc", ascending=False).head(10)

### Top 10 price to thd 1%, in 115.5 - 230.0 (thd_1pc_4ohm) category

In [ ]:
data[
    data["thd_category"] == "115.5 - 230.0"
].sort_values("price_to_thd_1pc", ascending=False).head(10)

### Top 10 price to thd 1%, in 230.0 - 422.5 (thd_1pc_4ohm) category

In [ ]:
data[
    data["thd_category"] == "230.0 - 422.5"
].sort_values("price_to_thd_1pc", ascending=False).head(10)

### Top 10 price to thd 1%, in 422.5 and above (thd_1pc_4ohm) category

In [ ]:
data[
    data["thd_category"] == "422.5 and above"
].sort_values("price_to_thd_1pc", ascending=False).head(10)

### Top 10 price to thd 1%, where sinad is >= 80

In [ ]:
data[
    data["sinad_5w_4ohm"] >= 80
].sort_values("price_to_thd_1pc", ascending=False).head(10)

### Top 10 price to thd 1%, where sinad is >= 100

In [ ]:
data[
    data["sinad_5w_4ohm"] >= 100
].sort_values("price_to_thd_1pc", ascending=False).head(10)

## Sinad analysis

### Top 20 sinad performance overall

In [ ]:
data.sort_values("sinad_5w_4ohm", ascending=False).head(20)

### Top 20 sinad per dollar performance, overall

In [ ]:
data.sort_values("price_to_sinad", ascending=False).head(20)

### Top 10 sinad per dollar performance, where sinad is > 80

In [ ]:
data[data["sinad_5w_4ohm"]  >= 80].sort_values("price_to_sinad", ascending=False).head(10)

### Top 10 sinad per dollar performance, where sinad is > 90

In [ ]:
data[data["sinad_5w_4ohm"]  >= 90].sort_values("price_to_sinad", ascending=False).head(10)

### Top 10 sinad per dollar performance, where sinad is > 100

In [ ]:
data[data["sinad_5w_4ohm"]  >= 100].sort_values("price_to_sinad", ascending=False).head(10)

## Price analysis

### Most expensive 20

In [ ]:
data.sort_values("price_usd", ascending=False).head(20)

### Cheapest 20

In [ ]:
data.sort_values("price_usd", ascending=True).head(20)

## Charts

### Sinad histogram

In [ ]:
data["sinad_5w_4ohm_integer"] = data["sinad_5w_4ohm"].apply(lambda x: int(x))

In [ ]:
histo_data = data.groupby(by="sinad_5w_4ohm_integer").size().reset_index()

In [ ]:
pygal_data  = [(histo_data.loc[x, 0], histo_data.loc[x, "sinad_5w_4ohm_integer"], histo_data.loc[x, "sinad_5w_4ohm_integer"] + 1) for x in list(histo_data.index)]

In [ ]:
chart = pygal.Histogram()
chart.config.title = f"Sinad histogram (all values rounded down to nearest integer). Mean: {round(data.sinad_5w_4ohm.mean(), 2)}"
chart.config.legend_at_bottom = True
chart.add('sinad_5w_4ohm',pygal_data)
# display(SVG(chart.render(disable_xml_declaration=True)))

# Export to svg
chart.render_to_file("./chart-exports/sinad-5watt-4ohm-histogram.svg")

### Price usd vs sinad scatter plot

In [ ]:
max_price_filter = 7500
first_quartile = data[(data.thd_category == "0 - 115.5") & (data.price_usd < max_price_filter)]
second_quartile = data[(data.thd_category == "115.5 - 230.0") & (data.price_usd < max_price_filter)]
third_quartile = data[(data.thd_category == "230.0 - 422.5") & (data.price_usd < max_price_filter)]
fourth_quartile = data[(data.thd_category == "422.5 and above") & (data.price_usd < max_price_filter)]

chart = pygal.XY(stroke=False)
chart.config.title = f"price usd (y axis) vs sinad 5watt 4ohm (x axis) where parts under ${max_price_filter} usd,  grouped by thd @ 1% category"
chart.config.legend_at_bottom = True

# Add data
chart.add("115.5 watts and under", list(zip(first_quartile.sinad_5w_4ohm, first_quartile.price_usd)))
chart.add("115.5 watts to 230 watts", list(zip(second_quartile.sinad_5w_4ohm, second_quartile.price_usd)))
chart.add("230 watts to 422.5 watts", list(zip(third_quartile.sinad_5w_4ohm, third_quartile.price_usd)))
chart.add("422.5 watts and above", list(zip(fourth_quartile.sinad_5w_4ohm, fourth_quartile.price_usd)))


# display(SVG(chart.render(disable_xml_declaration=True)))
# Export to svg
chart.render_to_file("./chart-exports/price-usd-vs-sinad-5watt-4ohm-scatterplot.svg")